In [69]:
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfVectorizer as tf
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("tweet_train.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [14]:
# We'll use this test set for cross validation
df = df[["id", "text", "target"]]
df = df.set_index("id")

In [15]:
df

,text,target
id,,
1,Our Deeds are the Reason of this #earthquake M...,1
4,Forest fire near La Ronge Sask. Canada,1
5,All residents asked to 'shelter in place' are ...,1
6,"13,000 people receive #wildfires evacuation or...",1
7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
10869,Two giant cranes holding a bridge collapse int...,1
10870,@aria_ahrary @TheTawniest The out of control w...,1
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1


In [16]:
target = df.pop("target")
attributes = df
X_train, X_cv, y_train, y_cv = train_test_split(attributes,target)

In [28]:
X_train

,text
id,
5855,Summer heat drives bobcats to Calgary backyard...
9739,Rly tragedy in MP: Some live to recount horror...
9142,Suicide bomber kills 15 in Saudi security site...
7231,Top insurer blasts lack of Australian Govt act...
3010,Adult dies of plague in Colorado http://t.co/y...
...,...
4673,michael is engulfed by that jumper
7809,Check Out: 'Reddit Will Now Quarantine Offensi...
2416,That sounds about right. Our building will hav...


In [70]:
vectorizer = tf()
X_train_vec = vectorizer.fit_transform(X_train["text"])
X_cv_vec = vectorizer.transform(X_cv["text"]) 

In [71]:
X_train_vec

<5709x17771 sparse matrix of type '<class 'numpy.float64'>'
	with 83644 stored elements in Compressed Sparse Row format>

In [72]:
X_train

,text
id,
5855,Summer heat drives bobcats to Calgary backyard...
9739,Rly tragedy in MP: Some live to recount horror...
9142,Suicide bomber kills 15 in Saudi security site...
7231,Top insurer blasts lack of Australian Govt act...
3010,Adult dies of plague in Colorado http://t.co/y...
...,...
4673,michael is engulfed by that jumper
7809,Check Out: 'Reddit Will Now Quarantine Offensi...
2416,That sounds about right. Our building will hav...


In [73]:
linear_svm = svm.SVC(kernel = "linear")
linear_svm.fit(X_train_vec, y_train)

SVC(kernel='linear')

In [74]:
linear_svm.score(X_cv_vec,y_cv)

0.8093487394957983

In [75]:
rbf_svm = svm.SVC(kernel = "rbf")
rbf_svm.fit(X_train_vec, y_train)

SVC()

In [76]:
rbf_svm.score(X_cv_vec,y_cv)

0.8135504201680672

In [77]:
sig_svm = svm.SVC(kernel = "sigmoid") 
sig_svm.fit(X_train_vec, y_train)

SVC(kernel='sigmoid')

In [78]:
sig_svm.score(X_cv_vec,y_cv)

0.8098739495798319

In [59]:
dftest = pd.read_csv("tweet_test.csv")
dftest.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [79]:
dfsub = pd.DataFrame()
dfsub["id"] = dftest["id"]
dfsub["target"] = rbf_svm.predict(vectorizer.transform(dftest["text"]))
dfsub

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [80]:
dfsub.to_csv("tweetsub.csv", index =False)